In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep, time
from random import uniform

In [2]:
input_file = "input.txt"

if input_file:
    # read keywords from file
    keywords = []
    with open(input_file, "r") as f:
        for line in f:
            tokens = line.split('*')
            if len(tokens) == 3:
                keywords.append(tokens[1])
else:
    # use default keywords
    keywords = ['subreddit', 'wikipedia']    
                
print(keywords)
print(len(keywords))

['heavy tail', 'running example', 'synthetic tasks', 'side information', 'tabular', 'qualification quiz', 'HIT', 'hit', '1-4 scale', '1-5 scale', 'Flesch', 'linearization', 'homogeneous', 'Hitlin', 'cold start', 'confidentiality', 'expertise', 'inter-training', 'cluster curriculum', 'curriculum cluster', 'stylistic distinction', 'sequential information bottleneck', 'sIB', 'normalized mutual information', 'NMI', 'pseudo-label MLM', 'pseudo-label BERT', 'key-point analysis', 'sparse patterns', 'locality sensitive hashing', 'LSH', 'partitioning', 'fuzzy clustering', 'blocks', 'finite experiments', 'task taxonomy', 'brain', 'neuroscience', 'brain activity', 'analytic hierarchy process', 'cognitive', 'exhaustive', 'transductive TL', 'inductive TL', 'lifelong learning', 'task embedding', 'Canberra distance', 'non-target', 'voxel', 'hierarchical clustering', 'long range', 'over-sensitive', 'oversensitive', 'recency bias', 'tempered sampling', 'log-linear boosting', 'undertraining', 'single pa

In [16]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# the returned number is unstable, need multiple clicks
clicks = 5

# the wait time needs to be randomized so the scraper is not identified as a bot
# the range is uniformly sampled between [a, b] seconds
# the time also needs to be large enough so the search results could be returned
a = 1
b = 1.5

# the output file, which will be manually edited after obtaining all output
fo = open("output.txt", "a")

# to start from midpoint, CHANGE THIS!
start_idx = 356

# with the current parameters, about 20 seconds per keyword
for keyword in keywords[start_idx:]:
    
    start = time()
    
    # random sleep for a few seconds
    sleep(uniform(5, 10))

    url = f"https://aclanthology.org/search/?q={keyword.replace(' ', '+')}"

    # start the driver
    driver.get(url)
    
    nums = []
    
    try:

        # switch between the options multiple times
        for _ in range(clicks):

            # expand the dropdown menu
            buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
            buttons[0].click()

            # choose year of publication
            buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
            buttons[1].click()

            # some time is needed for the result to change
            sleep(uniform(a,b))

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # result in the format "About 31 results (0.11 seconds)"
            result = soup.find_all("div", {"class": "gsc-result-info"})

            # get the number
            try:
                num = result[0].string.split(' ')[1].replace(',', '')
            except:
                num = 0
            nums.append(num)

            # expand the dropdown menu
            buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
            buttons[0].click()

            # change back to relevance
            buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
            buttons[0].click()
            sleep(uniform(a,b))

        end = time()

        print(keyword, min(nums), f"({end-start:.2f} seconds)")

        fo.write(f"{keyword}, {min(nums)}\n")
        fo.flush()
        
    except:
        print(f"The {keywords.index(keyword)}-th keyword {keyword} failed!")
        break

# quit the driver
driver.quit()

# close the file
fo.close()

fairseq noise function 0 (25.53 seconds)
continue finetuning 62 (21.63 seconds)
multi-task finetuning 280 (19.55 seconds)
span-span 980 (22.28 seconds)
subtree-subtree 104 (22.67 seconds)
left-out 51 (22.95 seconds)
expressivity 40 (23.33 seconds)
large margin objective 21 (25.05 seconds)
third-order 204 (21.72 seconds)
MST decoding 2 (21.16 seconds)
thorny 5 (21.89 seconds)
delexicalize 2 (20.32 seconds)
MoS 69 (23.79 seconds)
unwieldy 3 (24.42 seconds)
newswire 155 (23.89 seconds)
unassimilated 1 (22.06 seconds)
anglicism 2 (21.17 seconds)
anglicisms 2 (19.51 seconds)
Doccano 2 (23.74 seconds)
sense2vec 0 (20.71 seconds)
blenderbot finetune 0 (19.25 seconds)
KGAT 1 (20.80 seconds)
kernel graph attention network 2 (20.14 seconds)
proverbial 1 (21.98 seconds)
future predicting 72 (20.42 seconds)
n-stream 43 (22.20 seconds)
stream 638 (22.44 seconds)
Free Bits 4 (23.05 seconds)
full manual 69 (21.35 seconds)
entirely manual 41 (24.70 seconds)
gold-quality 297 (19.27 seconds)
synset 107 